In [55]:
import os
import json
import traceback
import pandas as pd
import langchain
import PyPDF2
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain


In [56]:
load_dotenv() 

True

In [57]:
key=os.getenv("google_api_key")

In [60]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=google_api_key, temperature=0.5)
llm

ChatGoogleGenerativeAI(model='gemini-pro', google_api_key=SecretStr('**********'), temperature=0.5, client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
))

In [61]:
with open('E:\Openai\MCQ\Response.json',"r") as f:
    RESPONSE_JSON = json.load(f)

In [66]:
print(RESPONSE_JSON)

{'1': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'mcq': 'multiple choice question', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [67]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [79]:
Quiz_generation_prompt=PromptTemplate(
    input_variables = ["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [80]:
quiz_chain=LLMChain(llm=llm, prompt=Quiz_generation_prompt, output_key="quiz", verbose=True)

In [94]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [98]:
Quiz_evaluation_prompt=PromptTemplate(input_variables = ["subject","quiz"],template=TEMPLATE2)

In [101]:
review_chain = LLMChain(llm=llm, prompt=Quiz_evaluation_prompt, output_key="review", verbose=True)

In [105]:
generate_evaulation_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","RESPONSE_JSON"], output_variables=['review'],verbose=True)

In [111]:
with open('E:\Openai\MCQ\data.txt', 'r') as file:
    text = file.read()
    print(text)


Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

Machine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised

In [114]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'